In [ ]:
API_KEY = 'v9j3zxdIsTKWmz0txOcFCpoiOqJfPoxd'

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def elastic_search(query: str) -> list:
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    return result_docs

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt: str, model: str) -> str | None:
    client = MistralClient(api_key=API_KEY)
    chat_completion = client.chat(
        messages=[ChatMessage(role="user", content=prompt)],
        model=model,
    )

    return chat_completion.choices[0].message.content

def rag(query: str, model: str = "mistral-large-latest") -> str | None:
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt, model)